## Magpie Toy Example

In [1]:
import transformers
import torch
import json
from transformers import AutoTokenizer
from fastchat.model import get_conversation_template

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, add_special_tokens=True)

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda:0",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Let's prepare the left-side template

In [2]:
with open("configs/model_configs.json", "r") as f:
    model_configs = json.load(f)
    model_config = model_configs[model_id]

# Prompt for extracting instructions from Llama-3-8B-Instruct
pre_query_template = model_config["pre_query_template"]
print(pre_query_template)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>




#### Step 1: Extracting Instructions

In [3]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

instruction = pipeline(
    pre_query_template,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=1,
)

sanitized_instruction = instruction[0]['generated_text'][len(pre_query_template):].split("\n")[0]
print(f"Extracted Instruction: {sanitized_instruction}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Extracted Instruction: I'd like to get a new pair of shoes for the summer. What kind of shoes are good for warm weather?


#### Step 2: Generating Responses

In [4]:
conv = get_conversation_template("llama-3")
print(conv) # Make sure the conversation template is correct with name='llama-3' !!!
conv.append_message(conv.roles[0], sanitized_instruction)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

response = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=1,
)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Conversation(name='llama-3', system_template='<|start_header_id|>system<|end_header_id|>\n\n{system_message}<|eot_id|>', system_message='', roles=('user', 'assistant'), messages=[], offset=0, sep_style=<SeparatorStyle.LLAMA3: 8>, sep='', sep2=None, stop_str='<|eot_id|>', stop_token_ids=[128001, 128009], max_image_size_mb=None)


### We have successfully created an instruction-response pair!

In [5]:
print(f"User's message: {sanitized_instruction}")
print(f"Assitant's response: {response[0]['generated_text'][len(prompt):]}")

User's message: I'd like to get a new pair of shoes for the summer. What kind of shoes are good for warm weather?
Assitant's response: Perfect timing! When it comes to shoes for warm weather, you'll want something comfortable, breathable, and stylish. Here are some great options:

1. **Sandals**: A classic choice for summer, sandals are perfect for walking, beach trips, or lounging around. Look for ones with good arch support and adjustable straps.
2. **Sneakers with breathable materials**: Sneakers made from mesh, canvas, or mesh panels are great for hot weather. Brands like New Balance, ASICS, or Converse offer such options.
3. **Espadrilles**: These flat, rope-soled shoes are perfect for casual outings. They're lightweight, breathable, and can be dressed up or down.
4. **Slide sandals**: Similar to traditional sandals, but with a more relaxed design. They're great for poolside or beach activities.
5. **Loafers**: A stylish option for warm weather, loafers are lightweight, comfortabl